In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from IPython.display import display

# install pkdb_analysis via:
#     git clone https://github.com/matthiaskoenig/pkdb_analysis (private)
#     cd pkdb_analysis
#     pip install -e .
# in future:
#     pip install pkdb_analsis

from pkdb_analysis import PKDB, PKData, PKFilter

WARNING Environment variables have not been initialized. 1. add authentication credentials; and 2. run set -a && source .env.local. Queries will be performed as a anonymous user.


# Filter data
A recurring task is to filter data for a certain question. E.g. to compare two groups, or get the subset of data for all healthy smokers.

We work again with our test data set and will filter various subsets from it.

In [2]:
test_data = PKDB.query()
TEST_HDF5 ="./test_data.hdf5"
#test_data.to_hdf5(path=TEST_HDF5)

INFO *** Querying data ***
INFO https://develop.pk-db.com/api/v1/interventions_analysis/?format=json&page_size=2000&normed=true
INFO https://develop.pk-db.com/api/v1/interventions_analysis/?format=json&page_size=2000&normed=true&page=1
INFO https://develop.pk-db.com/api/v1/individuals_analysis/?format=json&page_size=2000
INFO https://develop.pk-db.com/api/v1/individuals_analysis/?format=json&page_size=2000&page=1
INFO https://develop.pk-db.com/api/v1/individuals_analysis/?format=json&page_size=2000&page=2
INFO https://develop.pk-db.com/api/v1/groups_analysis/?format=json&page_size=2000
INFO https://develop.pk-db.com/api/v1/groups_analysis/?format=json&page_size=2000&page=1
INFO https://develop.pk-db.com/api/v1/output_analysis/?format=json&page_size=2000&normed=true
INFO https://develop.pk-db.com/api/v1/output_analysis/?format=json&page_size=2000&normed=true&page=1
INFO https://develop.pk-db.com/api/v1/output_analysis/?format=json&page_size=2000&normed=true&page=2
INFO https://develop.p

In [4]:
print(test_data)

------------------------------
PKData (140551171977936)
------------------------------
studies            44 
groups            147  ( 1078)
individuals       460  ( 3933)
interventions     137  (  157)
outputs          3096  ( 3096)
timecourses       296  (  296)
------------------------------


In [5]:
list(test_data.study_sids)[:10]

['PKDB00155',
 'PKDB00198',
 'PKDB00190',
 'PKDB00172',
 'PKDB00146',
 '24517114',
 'PKDB00195',
 'PKDB00222',
 'PKDB00184',
 'PKDB00193']

## Filter functions
The main principle for filtering `PKData` is by using the `filter_*` and `exclude_*` functionality.

A key principle are hereby filter functions which for a given DataFrame return a logical index.

Depending on which subset of information this should be applied the `groups`, `individuals`, `subjects` (groups and individuals), `outputs` and `timecourses`.

## Filter by `study_sid`
A first example is filtering by `study_sid`, i.e. we only want the subset of data from a single study.
An overview over the existing study sids in the dataset is available via

Filtering a subset of data works by providing filter/selection functions which select a subset of the data.
The filters are written on the `groups`, `individuals`

In [6]:
def is_PKDB99999(d):
    """Filter for specific study_sid. """
    return d.study_sid == "PKDB00198"

data = test_data.filter_intervention(is_PKDB99999)
print(data)

WARNING Concise DataFrames


------------------------------
PKData (140551143030416)
------------------------------
studies             1 
groups              4  (   36)
individuals        46  (  414)
interventions       1  (    1)
outputs            92  (   92)
timecourses         4  (    4)
------------------------------


The PKData now only contains data for the given study_sid:

In [7]:
print(data.study_sids)

{'PKDB00198'}


In [8]:
# for instance interventions
display(data.interventions)

,intervention_pk,study_sid,study_name,raw_pk,normed,name,route,form,application,time,...,substance,value,mean,median,min,max,sd,se,cv,unit
0,0,PKDB00198,Abernethy1982,1,True,paracetamol_iv,iv,solution,single dose,0.0,...,paracetamol,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram


One could also define this as a simple lambda function

In [9]:
data = test_data.filter_intervention(lambda d: d.study_sid == "PKDB00198")
print(data)

WARNING Concise DataFrames


------------------------------
PKData (140551141896400)
------------------------------
studies             1 
groups              4  (   36)
individuals        46  (  414)
interventions       1  (    1)
outputs            92  (   92)
timecourses         4  (    4)
------------------------------


## Concise data
All operations on `PKData` leave the data in a consistent manner. 
E.g. if an intervention is filtered out also all the outputs using this intervention are filtered out.
This behavior is controlled by the `concise` flag on most operations.

In [10]:
t1 = test_data.filter_intervention(is_PKDB99999)
t2 = test_data.filter_intervention(is_PKDB99999, concise=False)
print(t1)
print(t2)

WARNING Concise DataFrames


------------------------------
PKData (140551141476496)
------------------------------
studies             1 
groups              4  (   36)
individuals        46  (  414)
interventions       1  (    1)
outputs            92  (   92)
timecourses         4  (    4)
------------------------------
------------------------------
PKData (140551141525008)
------------------------------
studies            44 
groups            147  ( 1078)
individuals       460  ( 3933)
interventions       1  (    1)
outputs          3096  ( 3096)
timecourses       296  (  296)
------------------------------


In [11]:
# FIXME: only normed data
t1.interventions_mi

,study_sid,study_name,raw_pk,normed,name,route,form,application,time,time_end,...,substance,value,mean,median,min,max,sd,se,cv,unit
intervention_pk,,,,,,,,,,,,,,,,,,,,,
0,PKDB00198,Abernethy1982,1,True,paracetamol_iv,iv,solution,single dose,0.0,None,...,paracetamol,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram


In [12]:
t2.interventions_mi

,study_sid,study_name,raw_pk,normed,name,route,form,application,time,time_end,...,substance,value,mean,median,min,max,sd,se,cv,unit
intervention_pk,,,,,,,,,,,,,,,,,,,,,
0,PKDB00198,Abernethy1982,1,True,paracetamol_iv,iv,solution,single dose,0.0,None,...,paracetamol,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram


In [13]:
t2.outputs

,output_pk,intervention_pk,study_sid,study_name,group_pk,individual_pk,normed,calculated,tissue,time,...,substance,value,mean,median,min,max,sd,se,cv,unit
0,21,0,PKDB00198,Abernethy1982,3,-1,True,False,plasma,NaN,...,paracetamol,NaN,2.550000,NaN,1.9900,3.4700,NaN,NaN,NaN,hour
1,22,0,PKDB00198,Abernethy1982,3,-1,True,False,plasma,NaN,...,paracetamol,NaN,108.500000,NaN,62.2000,151.4000,NaN,NaN,NaN,liter
2,23,0,PKDB00198,Abernethy1982,3,-1,True,False,plasma,NaN,...,paracetamol,NaN,0.810000,NaN,0.5300,1.3100,NaN,NaN,NaN,liter / kilogram
3,24,0,PKDB00198,Abernethy1982,3,-1,True,False,plasma,NaN,...,paracetamol,NaN,29.040000,NaN,19.4400,38.7600,NaN,NaN,NaN,liter / hour
4,25,0,PKDB00198,Abernethy1982,3,-1,True,False,plasma,NaN,...,paracetamol,NaN,0.224400,NaN,0.1452,0.3156,NaN,NaN,NaN,liter / hour / kilogram
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3091,6428,136,PKDB00028,Iqbal1995,147,-1,True,True,plasma,NaN,...,paracetamol,NaN,0.018592,NaN,NaN,NaN,NaN,NaN,NaN,gram * hour / liter
3092,6429,136,PKDB00028,Iqbal1995,147,-1,True,True,plasma,NaN,...,paracetamol,NaN,0.004117,NaN,NaN,NaN,NaN,NaN,NaN,gram / liter
3093,6430,136,PKDB00028,Iqbal1995,147,-1,True,True,plasma,NaN,...,paracetamol,NaN,0.004558,NaN,NaN,NaN,NaN,NaN,NaN,1 / minute
3094,6431,136,PKDB00028,Iqbal1995,147,-1,True,True,plasma,NaN,...,paracetamol,NaN,2.534740,NaN,NaN,NaN,NaN,NaN,NaN,hour


## Query groups and individuals
### 2.1 Get data for groups with characteristica/keywords X
healthy=True, smoking=N, disease=None,
individual queries and combinations.


In [14]:
def is_healthy(d): 
    # healthy is reported and True
    return (d.measurement_type == "healthy") & (d.choice == "Y")

def disease(d):
    # any disease is reported
    return  d.measurement_type == "disease"

def smoking(d):
    # smoking status is curated for study (this could by Y/N/NR)
    return  d.measurement_type == "smoking"

def nonsmoker(d):
    # smoking is reported and no
    return smoking(d) & (d.choice == "N")

def smoker(d):
    # smoking is reported and yes
    return smoking(d) & (d.choice == "Y")

In [15]:
test_data = PKData.from_hdf5(TEST_HDF5)

NameError: name 'TEST_HDF5' is not defined

`f_idx` can be a single function, or a list of functions. A list of functions are applied successively and is equivalent to "AND logic". "OR logic" can be directly applied on the index.

In [16]:
healthy_nonsmoker = test_data.filter_subject(f_idx=[is_healthy, nonsmoker])
print(healthy_nonsmoker)
healthy_nonsmoker.groups_mi

WARNING Concise DataFrames


------------------------------
PKData (140551142472784)
------------------------------
studies            12 
groups             31  (  278)
individuals       162  ( 1445)
interventions      24  (   28)
outputs           697  (  697)
timecourses        44  (   44)
------------------------------


study_sid     study_name group_name  group_count  \
group_pk characteristica_pk                                                     
3        5                   PKDB00198  Abernethy1982  obese men            7   
         6                   PKDB00198  Abernethy1982  obese men            7   
         11                  PKDB00198  Abernethy1982  obese men            7   
         12                  PKDB00198  Abernethy1982  obese men            7   
         18                  PKDB00198  Abernethy1982  obese men            7   
...                                ...            ...        ...          ...   
132      2216                PKDB00216    Grattan2000        all           15   
         2217                PKDB00216    Grattan2000        all           15   
         2218                PKDB00216    Grattan2000        all           15   
         2219                PKDB00216    Grattan2000        all           15   
         2220                PKDB00216    Grattan2000        all           15   

                             group_parent_pk  count       measurement_type  \
group_pk characteristica_pk                                                  
3        5                                 2     42                species   
         6                                 2     42                healthy   
         11                                2     21          obesity index   
         12                                2     21    weight (categorial)   
         18                                2      7                    sex   
...                                      ...    ...                    ...   
132      2216                             -1     15                    bmi   
         2217                             -1     15         overnight fast   
         2218                             -1     15                smoking   
         2219                             -1     15             medication   
         2220                             -1     15  abstinence medication   

                                   choice substance  value  mean  median  \
group_pk characteristica_pk                                                
3        5                   homo sapiens      None    NaN   NaN     NaN   
         6                              Y      None    NaN   NaN     NaN   
         11                          None      None    NaN   NaN     NaN   
         12                         obese      None    NaN   NaN     NaN   
         18                             M      None    NaN   NaN     NaN   
...                                   ...       ...    ...   ...     ...   
132      2216                        None      None    NaN   NaN     NaN   
         2217                           Y      None    NaN   NaN     NaN   
         2218                           N      None    NaN   NaN     NaN   
         2219                           N      None    NaN   NaN     NaN   
         2220                        None      None    NaN   NaN     NaN   

                               min   max  sd  se  cv                   unit  
group_pk characteristica_pk                                                  
3        5                     NaN   NaN NaN NaN NaN                   None  
         6                     NaN   NaN NaN NaN NaN                   None  
         11                  133.0   NaN NaN NaN NaN                percent  
         12                    NaN   NaN NaN NaN NaN                   None  
         18                    NaN   NaN NaN NaN NaN                   None  
...                            ...   ...  ..  ..  ..                    ...  
132      2216                 19.0  30.0 NaN NaN NaN  kilogram / meter ** 2  
         2217                  NaN   NaN NaN NaN NaN                   None  
         2218                  NaN   NaN NaN NaN NaN                   None  
         2219                  NaN   NaN NaN NaN NaN                   None  
         2220                 30.0 

Often attributes are mixed for groups so we have to exclude the opposites.
In the example, the group `20` consists of 5 smokers and 1 nonsmoker. So for a subset of the group smoking is No.
We can exclude groups via

In [15]:
healthy_nonsmoker = test_data.filter_subject([is_healthy, nonsmoker]).exclude_subject([smoker])
print(healthy_nonsmoker)
display(healthy_nonsmoker.groups_mi)

WARNING Concise DataFrames
WARNING Concise DataFrames


------------------------------
PKData (140541914916368)
------------------------------
studies            77 
groups            114  ( 1081)
individuals       627  ( 5805)
interventions     157  (  258)
outputs          6012  ( 6012)
timecourses       389  (  389)
------------------------------


study_sid   study_name          group_name  \
group_pk characteristica_pk                                               
191      3444                PKDB00160  Adithan1982                 all   
         3445                PKDB00160  Adithan1982                 all   
         3446                PKDB00160  Adithan1982                 all   
         3447                PKDB00160  Adithan1982                 all   
         3448                PKDB00160  Adithan1982                 all   
...                                ...          ...                 ...   
789      15322               PKDB00112     Yue1989a  Chinese_M_NONSMOKE   
         15325               PKDB00112     Yue1989a  Chinese_M_NONSMOKE   
         15326               PKDB00112     Yue1989a  Chinese_M_NONSMOKE   
         15332               PKDB00112     Yue1989a  Chinese_M_NONSMOKE   
         15352               PKDB00112     Yue1989a  Chinese_M_NONSMOKE   

                             group_count  group_parent_pk  count  \
group_pk characteristica_pk                                        
191      3444                         10               -1     10   
         3445                         10               -1     10   
         3446                         10               -1     10   
         3447                         10               -1     10   
         3448                         10               -1     10   
...                                  ...              ...    ...   
789      15322                        84              780    133   
         15325                        84              780    132   
         15326                        84              780      1   
         15332                        84              780    110   
         15352                        84              780     84   

                                measurement_type        choice substance  \
group_pk characteristica_pk                                                
191      3444                            species  homo sapiens      None   
         3445                                sex             M      None   
         3446                                age          None      None   
         3447                             weight          None      None   
         3448                            healthy             Y      None   
...                                          ...           ...       ...   
789      15322                            weight          None      None   
         15325               oral contraceptives             N      None   
         15326               oral contraceptives             Y      None   
         15332                               sex             M      None   
         15352                           smoking             N      None   

                             value  mean  median   min   max   sd    se  cv  \
group_pk characteristica_pk                                                   
191      3444                  NaN   NaN     NaN   NaN   NaN  NaN   NaN NaN   
         3445                  NaN   NaN     NaN   NaN   NaN  NaN   NaN NaN   
         3446                  NaN  21.2     NaN   NaN   NaN  NaN  0.47 NaN   
         3447                  NaN  61.6     NaN   NaN   NaN  NaN  1.94 NaN   
         3448                  NaN   NaN     NaN   NaN   NaN  NaN   NaN NaN   
...                            ...   ...     ...   ...   ...  ...   ...  ..   
789      15322                 NaN  63.0     NaN  42.0  90.0  9.0   NaN NaN   
         15325                 NaN   NaN     NaN   NaN   NaN  NaN   NaN NaN   
         15326                 NaN   NaN     NaN   NaN   NaN  NaN   NaN NaN   
         15332                 NaN   NaN     NaN   NaN   NaN  NaN   NaN NaN   
         15352                 NaN   NaN     NaN   NaN   NaN  NaN   NaN NaN   

                                 unit  
group_pk characteristica_pk            
191      3444                    None  
         3445                  

In addition often combinations of attributes have to be used to find the correct subjects.
For instance a combination of `healthy` and reported `disease`

In [16]:
def is_healthy(d): 
    # healthy is reported and True
    return (d.measurement_type == "healthy") & (d.choice == "Y")

def disease(d):
    # any disease is reported
    return  d.measurement_type == "disease"

healthy1 = test_data.filter_subject(is_healthy)
healthy2 = test_data.exclude_subject(disease)
healthy3 = test_data.filter_subject(is_healthy).exclude_subject(disease)

print(healthy1)
print(healthy2)
print(healthy3)

WARNING Concise DataFrames
WARNING Concise DataFrames
WARNING Concise DataFrames
WARNING Concise DataFrames


------------------------------
PKData (140541898257104)
------------------------------
studies           198 
groups            334  ( 2935)
individuals      2164  (19140)
interventions     484  (  741)
outputs         17164  (17164)
timecourses      1220  ( 1220)
------------------------------
------------------------------
PKData (140541914805264)
------------------------------
studies           207 
groups            366  ( 3164)
individuals      2249  (19806)
interventions     503  (  760)
outputs         18416  (18416)
timecourses      1312  ( 1312)
------------------------------
------------------------------
PKData (140541914927504)
------------------------------
studies           197 
groups            334  ( 2935)
individuals      2144  (18991)
interventions     483  (  736)
outputs         17086  (17086)
timecourses      1220  ( 1220)
------------------------------


## 3 Query interventions
### 3.1 Get outputs/timecourses for intervention with substance
intervention with measurement_type "dosing" and substance "caffeine"

In [17]:
def dosing_and_caffeine(d):
    return ((d["measurement_type"]=="dosing") & (d["substance"]=="caffeine"))

In [18]:
test_data = PKData.from_hdf5(TEST_HDF5)

In [19]:
caffeine_data = test_data.filter_intervention(dosing_and_caffeine)

WARNING Concise DataFrames


In [20]:
print(caffeine_data)

------------------------------
PKData (140541760368528)
------------------------------
studies            63 
groups            102  (  954)
individuals       726  ( 6435)
interventions     110  (  207)
outputs          5747  ( 5747)
timecourses       330  (  330)
------------------------------


## 4 Query outputs/timecourses
### 4.1 query by measurement_type
filter all outputs with measurement_type auc_inf

In [21]:
def is_auc_inf(d):
    return (d["measurement_type"]=="auc_inf")  

test_data = PKData.from_hdf5(TEST_HDF5)

test_data = test_data.filter_output(is_auc_inf).delete_timecourses()
print(test_data)

WARNING Concise DataFrames
WARNING Concise DataFrames


------------------------------
PKData (140541761933392)
------------------------------
studies           155 
groups            200  ( 1713)
individuals       512  ( 4527)
interventions     336  (  529)
outputs          1877  ( 1877)
timecourses         0  (    0)
------------------------------


## 5 Other Query others
### 5.1 Complex
get clearance of codeine for all.h5 subjects, which have been phenotyped for cyp2d6. 


In [22]:
def is_cyp2d6_phenotyped(d):
    cyp2d6_phenotype_substances = ['spar/(2hspar+5hspar)', 'deb/4hdeb', 'dtf/dmt']
    return d["measurement_type"].isin(["metabolic phenotype", "metabolic ratio"]) & d["substance"].isin(cyp2d6_phenotype_substances)

def codeine_clearance(d):
    return (d["measurement_type"]=="clearance") & (d["substance"]=="codeine")                                                        

In [23]:
test_data = PKData.from_hdf5(TEST_HDF5)

phenotyped_data = test_data.filter_output(is_cyp2d6_phenotyped)

WARNING Concise DataFrames


In [24]:
test_data.groups = phenotyped_data.groups
test_data.individuals = phenotyped_data.individuals
test_data = test_data.filter_output(codeine_clearance).delete_timecourses()

WARNING Concise DataFrames
WARNING Concise DataFrames


In [25]:
print(test_data)

------------------------------
PKData (140541897941392)
------------------------------
studies            11 
groups             14  (  138)
individuals        13  (  135)
interventions      13  (   21)
outputs            30  (   30)
timecourses         0  (    0)
------------------------------
